In [24]:
import pandas as pd 
from tqdm import tqdm 
from collections import Counter
test=pd.read_csv('/kaggle/input/sentiment-analysis-dataset/test.csv',encoding='latin1')
test = test.dropna()
test

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0
...,...,...,...,...,...,...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative,noon,21-30,Nicaragua,6624554.0,120340.0,55.0
3530,416863ce47,All alone in this old house again. Thanks for...,positive,night,31-45,Niger,24206644.0,1266700.0,19.0
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative,morning,46-60,Nigeria,206139589.0,910770.0,226.0
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive,noon,60-70,North Korea,25778816.0,120410.0,214.0


In [25]:
texts=test['text']
sentiments=test['sentiment']
len(texts),len(sentiments)

(3534, 3534)

In [26]:
from transformers import pipeline
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

sentipipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment", device=device)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [28]:
preds=[]
for text in tqdm(texts):
    sentiment = sentipipe(text)
    sentiment_mapping = {
            "LABEL_0": "negative",
            "LABEL_1": "neutral",
            "LABEL_2": "positive"
        }
    sentiment[0]['label'] = sentiment_mapping[sentiment[0]['label']]
    preds.append(sentiment[0]['label'])

100%|██████████| 3534/3534 [00:31<00:00, 111.47it/s]


In [29]:
len(preds)

3534

In [30]:
from sklearn.metrics import classification_report

# True labels (ground truth)
y_true = sentiments

# Predicted labels
y_pred = preds

# Generate the classification report
report = classification_report(y_true, y_pred)

print(report)


              precision    recall  f1-score   support

    negative       0.70      0.79      0.74      1001
     neutral       0.74      0.56      0.63      1430
    positive       0.72      0.86      0.78      1103

    accuracy                           0.72      3534
   macro avg       0.72      0.74      0.72      3534
weighted avg       0.72      0.72      0.71      3534

